# QCEW Filtration Data Kit

## Introduction

This Jupyter Notebook is designed to filter Quarterly Census of Employment and Wages (QCEW) datasets, retreived from [qcew_data_kit_2](https://github.com/olivergwynn/qcew_data_kit/tree/main). The dataset(s) processed by this notebook saves a long-form file data for each variable, with a variety of identifying variables, including year, industry, and county.

## Process Outline

The process carried out by this workflow can be described as follows:
  - Summarized annual and/or quarterly long-form QCEW files in input directory are split for each measured variable, and each variable is saved as stand-alone '.csv' files.
  - For each processed long-from '.csv', a .resource.yaml file is created, following the Frictionless Data Resource specification. This file includes metadata about the CSV file, such as its name, path, format, and the schema it conforms to, as well as a hash code for integrity checking. Additionally, it contains descriptive information about the dataset and references to its source
  - The YAML files for schemas and resource descriptors are used to make data more usable by simplifying its publication and consumption. By adhering to Frictionless standards, the script ensures that the datasets it produces are easily shareable, validatable, and integrable into a wide range of data tools and platforms.

## Setup

### Import required packages

In [8]:
import pandas as pd
import os
import frictionless
import sys
import hashlib
sys.path.append(os.path.normpath("../../morpc-common"))
import morpc

### Parameters

#### Static parameters

In [9]:
Dtypes = {
    'disclosure_code': 'str',
    'industry_code': 'str',
    'lq_disclosure_code': 'str',
    'oty_disclosure_code': 'str'
}

OUTPUT_DIR = os.path.normpath("./output_data")
INPUT_DIR = os.path.normpath("./input_data")

# File name for long-form quarterly table
QCEW_QUARTERLY_LONG_INPUT_NAME  = "qcew_quarterly_long_summarized.csv" 
QCEW_ANNUAL_LONG_INPUT_NAME  = "qcew_annual_long_summarized.csv" 

# Quarterly data paths
QCEW_QUARTERLY_LONG_INPUT_PATH = os.path.join(INPUT_DIR, QCEW_QUARTERLY_LONG_INPUT_NAME)
QCEW_ANNUAL_LONG_INPUT_PATH = os.path.join(INPUT_DIR, QCEW_ANNUAL_LONG_INPUT_NAME)

LONG_TABLE_SCHEMA_FILENAME = "morpc-qcew-long.schema.yaml"
LONG_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, LONG_TABLE_SCHEMA_FILENAME)

### Define inputs

In [10]:
print("Long schema file stored in: {}".format(LONG_TABLE_SCHEMA_PATH))
print("Summarized quarterly long-form QCEW data to be summarized must be stored in: {}".format(QCEW_QUARTERLY_LONG_INPUT_PATH))
print("Summarized annual long-form QCEW data to be summarized must be stored in: {}".format(QCEW_ANNUAL_LONG_INPUT_PATH))

Long schema file stored in: output_data\morpc-qcew-long.schema.yaml
Summarized quarterly long-form QCEW data to be summarized must be stored in: input_data\qcew_quarterly_long_summarized.csv
Summarized annual long-form QCEW data to be summarized must be stored in: input_data\qcew_annual_long_summarized.csv


### Define output

In [11]:
print("Filtered annual data files will be saved as: {}".format(os.path.join(OUTPUT_DIR, "qcew_annual_long_summarized_{variable}.csv")))
print("Filtered annual resource files will be saved as: {}".format(os.path.join(OUTPUT_DIR, "qcew_annual_long_summarized_{variable}.resource.yaml")))
print("Filtered quarterly data files will be saved as: {}".format(os.path.join(OUTPUT_DIR, "qcew_annual_long_summarized_{variable}.csv")))
print("Filtered quarterly resource files will be saved as: {}".format(os.path.join(OUTPUT_DIR, "qcew_annual_long_summarized_{variable}.resource.yaml")))

Filtered annual data files will be saved as: output_data\qcew_annual_long_summarized_{variable}.csv
Filtered annual resource files will be saved as: output_data\qcew_annual_long_summarized_{variable}.resource.yaml
Filtered quarterly data files will be saved as: output_data\qcew_annual_long_summarized_{variable}.csv
Filtered quarterly resource files will be saved as: output_data\qcew_annual_long_summarized_{variable}.resource.yaml


### Method to create resource file and validate long-form outputs each variable

In [12]:
def is_numeric(val):
    """Check if the value is numeric."""
    try:
        float(val)
        return True
    except ValueError:
        return False

def process_qcew_data(path):
    df_name = os.path.basename(path)
    df_resource_path = path.replace(".csv", ".resource.yaml")
    df = pd.read_csv(path, dtype=Dtypes)

    # Parsing data for naming convention
    max_year = df['year'].max()
    min_year = df['year'].min()
    variable_name = df['variable'].iloc[0] 
    # Determine if the data is annual or quarterly
    if is_numeric(df['variable'].iloc[0]):
        period_type = 'quarterly'
    else:
        period_type = 'annual'

    # Update title and description with the county name
    title = f"Compiled QCEW County Data, {period_type.capitalize()}, {min_year}-{max_year} (wide form), Variable: {variable_name}"
    description = f"Variable: {variable_name} data for counties in MOPRC region and summary {min_year}-{max_year}, derived from the U.S. Bureau of Labor Statistics. Data period: {period_type.capitalize()}."

    # Resource creation for WIDE QUARTERLY
    if not df.empty:
        acsResource = {
            "name": f"qcew_{period_type}_{min_year}_{max_year}_{variable_name}",
            "title": title,
            "description": description,
            "path": df_name,
            "format": "csv",
            "mediatype": "text/csv",
            "encoding": "utf-8",
            "bytes": os.path.getsize(path),
            "hash": morpc.md5(path),
            "schema": LONG_TABLE_SCHEMA_FILENAME,
            "profile": 'tabular-data-resource'
        }

        # Create the resource object
        resource = frictionless.Resource(acsResource)

        print("Writing resource file to {}".format(df_resource_path))
        cwd = os.getcwd()
        os.chdir(os.path.dirname(df_resource_path))
        dummy = resource.to_yaml(os.path.basename(df_resource_path))
        os.chdir(cwd)

        print("Validating resource on disk (including data and schema). This may take some time.")
        resourceOnDisk =  frictionless.Resource(df_resource_path)
        results = resourceOnDisk.validate()
        if results.valid:
            print("Resource is valid\n")
        else:
            print("ERROR: Resource is NOT valid. Errors follow.\n")
            print(results)
            raise RuntimeError("Resource validation failed.")


## Filtering annual QCEW data

In [13]:
# Check if the file exists and is not empty
if os.path.exists(QCEW_ANNUAL_LONG_INPUT_PATH) and os.path.getsize(QCEW_ANNUAL_LONG_INPUT_PATH) > 0:
    # Load the CSV file
    df_annual = pd.read_csv(QCEW_ANNUAL_LONG_INPUT_PATH, dtype=Dtypes)

    if not df_annual.empty:
        # Get the base name of the input file (without extension)
        base_name = os.path.splitext(os.path.basename(QCEW_ANNUAL_LONG_INPUT_PATH))[0]

        # Group by 'variable' and save each group to a separate CSV file
        for variable, group in df_annual.groupby('variable'):
            output_file = os.path.join(OUTPUT_DIR, f'{base_name}_{variable}.csv')
            group.to_csv(output_file, index=False)
            process_qcew_data(output_file)
    else:
        print("No annual data to melt")
else:
    print(f"{QCEW_ANNUAL_LONG_INPUT_PATH} does not exist or is empty.")

Writing resource file to output_data\qcew_annual_long_summarized_annual_avg_emplvl.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_annual_avg_estabs.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_annual_avg_wkly_wage.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_annual_contributions.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_avg_annual_pay.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing

## Filtering quarterly QCEW data

In [14]:
# Check if the file exists and is not empty
if os.path.exists(QCEW_QUARTERLY_LONG_INPUT_PATH) and os.path.getsize(QCEW_QUARTERLY_LONG_INPUT_PATH) > 0:
    # Load the CSV file
    df_quarterly = pd.read_csv(QCEW_QUARTERLY_LONG_INPUT_PATH, dtype=Dtypes)

    if not df_quarterly.empty:
        # Get the base name of the input file (without extension)
        base_name = os.path.splitext(os.path.basename(QCEW_ANNUAL_LONG_INPUT_PATH))[0]

        # Group by 'variable' and save each group to a separate CSV file
        for variable, group in df_quarterly.groupby('variable'):
            output_file = os.path.join(OUTPUT_DIR, f'{base_name}_{variable}.csv')
            group.to_csv(output_file, index=False)
            process_qcew_data(output_file)
    else:
        print("No quarterly data to melt")
else:
    print(f"{QCEW_QUARTERLY_LONG_INPUT_PATH} does not exist or is empty.")

Writing resource file to output_data\qcew_annual_long_summarized_avg_wkly_wage.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_lq_avg_wkly_wage.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_lq_month1_emplvl.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_lq_month2_emplvl.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource file to output_data\qcew_annual_long_summarized_lq_month3_emplvl.resource.yaml
Validating resource on disk (including data and schema). This may take some time.
Resource is valid

Writing resource f